# Project Football

In this project we checked historical data about football matches.

_SOL_: I made a change about some business questions.

## Data Collection

In [ ]:
# consume external API

In [ ]:
# make web scrapping to get stats from Wikipedia

In [5]:
import pandas as pd

# get data from kaggle
football_df = pd.read_csv('./data/football.csv')
football_df.columns.to_list()

/tmp/ipykernel_9339/1709806642.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  football_df = pd.read_csv('./data/football.csv')


['id',
 'target',
 'home_team_name',
 'away_team_name',
 'match_date',
 'league_name',
 'league_id',
 'is_cup',
 'home_team_coach_id',
 'away_team_coach_id',
 'home_team_history_match_date_1',
 'home_team_history_match_date_2',
 'home_team_history_match_date_3',
 'home_team_history_match_date_4',
 'home_team_history_match_date_5',
 'home_team_history_match_date_6',
 'home_team_history_match_date_7',
 'home_team_history_match_date_8',
 'home_team_history_match_date_9',
 'home_team_history_match_date_10',
 'home_team_history_is_play_home_1',
 'home_team_history_is_play_home_2',
 'home_team_history_is_play_home_3',
 'home_team_history_is_play_home_4',
 'home_team_history_is_play_home_5',
 'home_team_history_is_play_home_6',
 'home_team_history_is_play_home_7',
 'home_team_history_is_play_home_8',
 'home_team_history_is_play_home_9',
 'home_team_history_is_play_home_10',
 'home_team_history_is_cup_1',
 'home_team_history_is_cup_2',
 'home_team_history_is_cup_3',
 'home_team_history_is_cup_

In [ ]:
# merge data sources in one or multiple dataframes

In [ ]:
# plot histogram of the data

# plot heatmap about correlations

## Data Preprocessing

### Data Cleaning

We remove, fill, replace, ...

In [ ]:
# code to remove duplicates

# code to remove missing values

# code to remove outliers



In [ ]:
# plot before and after remove outliers and clean data

### Data Preparation

In [ ]:
# change datetime format

# change columns data type

# convert dtetime to unix timestamp

## Feature Engineering


In [ ]:
# create columns by month based on date

# one hot encoding for categorical variables

# get a new column based on a condition over other columns

# get categorical column based on numerical column

## Model Selection

In [ ]:
# Split into test dataset and train dataset

### Test Linear Regression


In [ ]:
# linear regresion for blah blah

### Test K-Nearest Neighbors

In [ ]:
# Test KKN for classification

### Test XGBoost for Regression

In [ ]:
# code for a XGBoost regressor

### Models Comparison

In [ ]:
# compare metrics

## Model Training

In [ ]:
# prepare data for the model

# scaling data - additional encoder

# feature selection

# create model with hyperparameters

# train model

## Model Evaluation

In [ ]:
# test metrics classical

# show plots about results

In [ ]:
# scatter plot comparing real and predicted values

# line plot about real vs predicted

# box plot about real vs predicted

In [ ]:
# pickle the model
import pickle

# save the model to disk
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

***

# Business Questions

1. What is the risk level for unpayments based on age and region?


In [1]:
# generate a table or a plot or a business metric

2. What is average amount of unpayments? what is the segment of values where unpayments are appearin?

In [ ]:
# generate a table or a plot or a business metric